In [1]:
import pandas as pd 
from datetime import date
from datetime import timedelta

In [7]:
from pandasql import sqldf

In [ ]:
df = pd.read_csv('/path/to/file.csv') #put the path to your  csv here
ndf = df[['CURRENT_BADGE_NO','OCCURRED_DT']] #create a dataframe with just a unique identifier (badge number) and the date that the allegation
ndf

In [9]:
ndf = ndf.sort_values(['CURRENT_BADGE_NO', 'OCCURRED_DT']) #sort by badge number and occured date
ndf

NameError: name 'ndf' is not defined

In [91]:
ndf = ndf.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False) #drop  all nan values
ndf

,CURRENT_BADGE_NO,OCCURRED_DT
10884,0,1986-07-02
20844,0,1986-08-06
20870,0,1986-11-02
20878,0,1986-11-21
56635,0,1987-02-27
...,...,...
18217,T273,2017-12-28
19004,T282,2018-11-15
19356,T282,2018-12-09
18391,T296,2018-02-01


In [11]:
ndf = ndf[ndf.OCCURRED_DT != 'Protected Data'] #drop protected data
ndf

NameError: name 'ndf' is not defined

In [12]:
#query to group the dataframe by badge number AND occured date so we dont get two allegations on the same date, which could potentially screw up the average
q="""SELECT CURRENT_BADGE_NO, OCCURRED_DT FROM ndf GROUP BY CURRENT_BADGE_NO, OCCURRED_DT ORDER BY CURRENT_BADGE_NO, OCCURRED_DT;"""
pysqldf = lambda q: sqldf(q, globals())
ndf = pysqldf(q)

PandaSQLException: (sqlite3.OperationalError) no such table: ndf
[SQL: SELECT CURRENT_BADGE_NO, OCCURRED_DT FROM ndf GROUP BY CURRENT_BADGE_NO, OCCURRED_DT ORDER BY CURRENT_BADGE_NO, OCCURRED_DT;]
(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [10]:
ndf

NameError: name 'ndf' is not defined

In [95]:
cbns = [] #create an empty list for badge numbers
cops = [] #create an empty list for badge number + occured date
for cbn, rdt in zip(ndf['CURRENT_BADGE_NO'], ndf['OCCURRED_DT']): 
    if cbn not in cbns: #if not already in the list
        cbns.append(cbn) #add it to the list
    x = [cbn,rdt] #define a variable with each badge number and occured date
    cops.append(x) #save that to the other list

In [98]:
dicts = {} #convert to dictionary
for i in cops:
    x = []
    for j in cops: #loop through the cops list twice 
        if i[0] == j[0]:
            x.append(date.fromisoformat(j[1])) #append the date to that cops badge number
            x = sorted(x) #sort the dates chronologically
        else: 
            pass
    dicts.update({(i[0]): x}) #create the new dictionary entry

In [13]:
print(dicts)

NameError: name 'dicts' is not defined

In [14]:
for j,k in dicts.items(): #loop through items in dict
    y = timedelta(days=0) #create a empty/null timedelta
    for i in range(1,len(k)):
        x = k[i] - k[i-1] #date minus date before it
        y += x #sum up all differences
        n = len(k) - 1 #divided by number of values
        score = y/n #compute the average timedelta
        dicts.update({j:score}) #update the dict with the average time between allegations

print(dicts)

NameError: name 'dicts' is not defined

In [15]:
ldf = pd.DataFrame(list(dicts.items()),columns = ['badge_no','score']) #convert from dictionary to dataframe
ldf

NameError: name 'dicts' is not defined

In [86]:
ldf.to_csv('score.csv', index=False) #save as csv